In [19]:
# imports
import requests
import pandas as pd
import time

In [193]:
#Taken from city_bikes notebook. 

import requests
import pandas as pd

detroit_url = "http://api.citybik.es/v2/networks/mogo-detroit"
detroitresponse = requests.get(detroit_url)

if detroitresponse.status_code == 200:
    detroitdata = detroitresponse.json()
    detroitstations = detroitdata["network"]["stations"]
    
    detroitbikes = pd.DataFrame([{
        "station_id": station["id"],
        "name": station["name"],
        "latitude": station["latitude"],
        "longitude": station["longitude"],
        "free_bikes": station["free_bikes"],
        "empty_slots": station["empty_slots"]
    } for station in detroitstations])
    
    print(detroitbikes.head())

                         station_id                       name   latitude  \
0  019fbdf044ddeded277660f9d9c5fb23    E Bethune St & Brush St  42.374231   
1  021d0e9c8fcd21ec32bd3136945ef332        20th St & Bagley St  42.324627   
2  0ad79e03add4ffef597d9a7b51d374be       Agnes St & Parker St  42.355351   
3  0d077ef68550428f754b9685b5750b27   Wabash St & Michigan Ave  42.331483   
4  0d7d5ce52fd3c9eea183caed919e0dd1  *N Main St & Gardenia Ave  42.497780   

   longitude  free_bikes  empty_slots  
0 -83.069759           1           10  
1 -83.080853           5            5  
2 -82.996276           5            9  
3 -83.075110           5            6  
4 -83.144430           4            3  


In [23]:
foursquare_api_key = "fsq38YB6itlntx2lurgkGxiW9XqQjcWUihwNUVVqAkUoPTc="
yelp_api_key = "07rbnsS52UCUMx7lw818kBL_tyFz91JxScYD1zzZvouGHBgBC6-MtavklR8Nr0eMTpwKMte5j7L8QmiTpL6hM1RGxwOdWptDugN8I0BHbGCbKaE7-8otVXI0I9lQaHYx"

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [64]:
#I started by defining the headers to make requests 
foursquare_headers = {
    "Accept": "application/json",
    "Authorization": foursquare_api_key
}

#Created a function to get specific Points of Interest 
def foursquare_POI (lat, lon, radius=1000, categories = ["13065", "13003", "13032"]
):  
    #Set the endpoint & construct a query 
    url = "https://api.foursquare.com/v3/places/search"
    params = {
        "ll": f"{lat},{lon}",
        "radius": radius,
        "categories": ",".join(categories),
        "limit": 20
    }

    #GET request was sent to Foursquare Places API
    response = requests.get(url, headers=foursquare_headers)

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [195]:
import time

#Create an empty list to later store in the POI Data
foursquare_POIdata = []

#Loop through ever row in the df
#Each row= 1 bike station 
#iterrows iterates each row
for _, row in detroitbikes.iterrows():
    #Returns POI's in a 1000m radius 
    POIs = get_foursquare_pois(row["latitude"], row["longitude"])
    print(json.dumps(POIs, indent=2))
        
    #The following were extracted for every POI detected
    for POI in POIs:
        foursquare_POIdata.append({
            "station_id": row["station_id"],
            "POI_name": poi.get("name"),
            "POI_lat": poi["geocodes"]["main"]["latitude"],
            "POI_lon": poi["geocodes"]["main"]["longitude"],
            "category": poi["categories"][0]["name"] if poi.get("categories") else None,
            "address": poi.get("location", {}).get("formatted_address")
        })
    
    # Pausing for atleast a second to ensure not too many API requests are not sent at once
    time.sleep(1)

print(foursquare_poi_data[:3])

#At the end of this loop, I have a list that holds foursquare POI data near each Detroit bike station

[
  {
    "fsq_id": "5a557e0828122f3fc232d5de",
    "categories": [
      {
        "id": 13009,
        "name": "Cocktail Bar",
        "short_name": "Cocktail",
        "plural_name": "Cocktail Bars",
        "icon": {
          "prefix": "https://ss3.4sqi.net/img/categories_v2/nightlife/cocktails_",
          "suffix": ".png"
        }
      },
      {
        "id": 13065,
        "name": "Restaurant",
        "short_name": "Restaurant",
        "plural_name": "Restaurants",
        "icon": {
          "prefix": "https://ss3.4sqi.net/img/categories_v2/food/default_",
          "suffix": ".png"
        }
      }
    ],
    "chains": [],
    "closed_bucket": "LikelyOpen",
    "distance": 393,
    "geocodes": {
      "drop_off": {
        "latitude": 42.371165,
        "longitude": -83.067011
      },
      "main": {
        "latitude": 42.371308,
        "longitude": -83.067063
      },
      "roof": {
        "latitude": 42.371308,
        "longitude": -83.067063
      }
    },
    "

KeyError: 'geocodes'

Put your parsed results into a DataFrame

In [76]:
foursquaredf = pd.DataFrame(foursquare_POIdata)
print(foursquaredf.head())

                         station_id   POI_name   POI_lat    POI_lon  \
0  019fbdf044ddeded277660f9d9c5fb23  Tuff Taco  42.34424 -83.157737   
1  019fbdf044ddeded277660f9d9c5fb23  Tuff Taco  42.34424 -83.157737   
2  019fbdf044ddeded277660f9d9c5fb23  Tuff Taco  42.34424 -83.157737   
3  019fbdf044ddeded277660f9d9c5fb23  Tuff Taco  42.34424 -83.157737   
4  019fbdf044ddeded277660f9d9c5fb23  Tuff Taco  42.34424 -83.157737   

     category                                 address  
0  Restaurant  10005 W Warren Ave, Dearborn, MI 48126  
1  Restaurant  10005 W Warren Ave, Dearborn, MI 48126  
2  Restaurant  10005 W Warren Ave, Dearborn, MI 48126  
3  Restaurant  10005 W Warren Ave, Dearborn, MI 48126  
4  Restaurant  10005 W Warren Ave, Dearborn, MI 48126  


# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [165]:
#Defining the headers to make requests 
yelp_headers = {
    "Authorization": f"Bearer {yelp_api_key}"
}

#Created a function to get specific Points of Interest 
def get_yelp_POIs(lat, lon, radius=1000, term="restaurant"):
    url = "https://api.yelp.com/v3/businesses/search"
    params = {
        "term": term,
        "latitude": lat,
        "longitude": lon,
        "radius": radius,
        "limit": 20,
        "categories": "restaurants,bars"
    }

#The cell below was having issues running. The following aided in trying to solve the issue
    try:
        #Sends a GET request to the Yelp API
        response = requests.get(url, headers=yelp_headers, params=params)
        #Ensures failed response does not get parsed 
        response.raise_for_status()
        data = response.json()
        return data.get("businesses", [])
        #Catches exception raised in the block
    except Exception as e:
        return []

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [168]:
#Create an empty list to later store in the POI Data
yelp_POIdata = []

#Loop through ever row in the df
#Each row= 1 bike station 
#iterrows iterates each row
for _, row in detroitbikes.iterrows():
    #Returns POI's in a 1000m radius
    yelp_POIs = get_yelp_POIs(row["latitude"], row["longitude"])

    #The following were extracted for every POI detected
    for POI in yelp_POIs:
        yelp_POIdata.append({
            "station_id": row["station_id"],
            "POI_name": POI.get("name"),
            "rating": POI.get("rating"),
            "review_count": POI.get("review_count"),
            "POI_lat": POI.get("coordinates", {}).get("latitude"),
            "POI_lon": POI.get("coordinates", {}).get("longitude"),
            "category": POI["categories"][0]["title"] if POI.get("categories") else None,
            "address": ", ".join(POI.get("location", {}).get("display_address", []))
        })
    
    # Pausing for atleast a second to ensure not too many API requests are not sent at once
    time.sleep(1)

#At the end of this loop, I have a yelp list that holds POI data near each Detroit bike station

In [135]:
yelp_df = pd.DataFrame(yelp_POIdata)
yelp_df.head()

,station_id,POI_name,rating,review_count,POI_lat,POI_lon,category,address
0,019fbdf044ddeded277660f9d9c5fb23,Oak & Reel,4.5,288,42.372278,-83.066819,Italian,"2921 E Grand Blvd, Detroit, MI 48202"
1,019fbdf044ddeded277660f9d9c5fb23,Freya,4.4,110,42.372169,-83.066983,New American,"2929 E Grand Blvd, Detroit, MI 48202"
2,019fbdf044ddeded277660f9d9c5fb23,See You Tomorrow,4.2,60,42.373968,-83.075200,Breakfast & Brunch,"7760 Woodward Ave, Detroit, MI 48226"
3,019fbdf044ddeded277660f9d9c5fb23,Baobab Fare,4.7,367,42.370077,-83.072813,Coffee & Tea,"6568 Woodward Ave, Detroit, MI 48202"
4,019fbdf044ddeded277660f9d9c5fb23,Choixes,2.5,16,42.375141,-83.077095,Juice Bars & Smoothies,"8201 Woodward Ave, Detroit, MI 48202"


Put your parsed results into a DataFrame

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Between the Foursquare and Yelp APIs, Yelp offers more complete data. 

Foursquare provides basic info like the name, address, and location of places, but does not provide ratings nor reviews. It’s may be ideal for dicovering places, but not for assessing quality. 

Yelp  provides ratings, review counts, and more detailed business info, making it easier for evaluating places. Since Yelp is more focused on the informational aspect, it would be the better choice when you need thorough data on location quality.

Get the top 10 restaurants according to their rating

To get the top 10 restaurants based on their ratings, Yelp would be the primary choice. As mentioned, it provides ratings and reviews, making it easy to rank restaurants. Foursquare doesn’t include ratings, so it’s not ideal for this task. 

In [191]:
#Sorts the yelp POI list in descending order, to achieve the top 10 restaurants 
for poi in sorted(yelp_POIs, key=lambda x: x["rating"], reverse=True)[:10]: print(f"{poi['name']} - {poi['rating']}")

El Takito Loko - 5.0
Annie Mae's Kitchen - 5.0
Dem Noles Bois - 5.0
Zaatar W Zeit - 4.7
Asty Time Dominican Restaurant - 4.6
El Rey de Las Arepas - 4.5
Tuhama's - 4.3
The Dog House - 4.3
Farooj - 4.0
Shottys Kitchen Halal - 3.8


In [201]:
foursquaredf.to_csv("foursquare_data.csv", index=False)
yelp_df.to_csv("yelp_data.csv", index=False)
